In [1]:
import os,sys
sys.path.append('../')
import camb
import hmvec as hm
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
import time

In [3]:
# Create model
zs = np.linspace(0.,3.,20)  # 20
ms = np.geomspace(2e10,1e17,200)  # 200
ks = np.geomspace(1e-4,100,1001)  # 1001
hcos = hm.HaloModel(zs,ks,ms=ms)

xmax = 20  # 20
nxs = 5000  # 5000

In [2]:
def add_battaglia_profile(self,name,family=None,param_override=None,
                              nxs=None,
                              xmax=None,ignore_existing=False):
        if not(ignore_existing): assert name not in self.uk_profiles.keys(), "Profile name already exists."
        assert name!='nfw', "Name nfw is reserved."
        if nxs is None: nxs = self.p['electron_density_profile_integral_numxs']
        if xmax is None: xmax = self.p['electron_density_profile_integral_xmax']

        
        # Set default parameters
        if family is None: family = self.p['battaglia_gas_family'] # AGN or SH?
        pparams = {}
        pparams['battaglia_gas_gamma'] = self.p['battaglia_gas_gamma']
        pparams.update(battaglia_defaults[family])

        # Update with overrides
        if param_override is not None:
            print(param_override)
            for key in param_override.keys():
                if key=='battaglia_gas_gamma':
                    pparams[key] = param_override[key]
                elif key in battaglia_defaults[family]:
                    pparams[key] = param_override[key]
                else:
                    #raise ValueError # param in param_override doesn't seem to be a Battaglia parameter
                    pass

        # Convert masses to m200critz
        rhocritz = self.rho_critical_z(self.zs)
        if self.mdef=='vir':
            delta_rhos1 = rhocritz*self.deltav(self.zs)
        elif self.mdef=='mean':
            delta_rhos1 = self.rho_matter_z(self.zs)*200.
        rvirs = self.rvir(self.ms[None,:],self.zs[:,None])
        cs = self.concentration()
        delta_rhos2 = 200.*self.rho_critical_z(self.zs)
        m200critz = mdelta_from_mdelta(hcos.ms,cs,delta_rhos1,delta_rhos2)
        r200critz = R_from_M(m200critz,hcos.rho_critical_z(hcos.zs)[:,None],delta=200.)

        # Generate profiles
        """
        The physical profile is rho(r) = f(2r/R200)
        We rescale this to f(x), so x = r/(R200/2) = r/rgs
        So rgs = R200/2 is the equivalent of rss in the NFW profile
        """
        omb = hcos.p['ombh2'] / hcos.h**2.
        omm = hcos.om0
        rhofunc = lambda x: rho_gas_generic_x(x,m200critz[...,None],hcos.zs[:,None,None],omb,omm,rhocritz[...,None,None],
                                    gamma=pparams['battaglia_gas_gamma'],
                                    rho0_A0=pparams['rho0_A0'],
                                    rho0_alpham=pparams['rho0_alpham'],
                                    rho0_alphaz=pparams['rho0_alphaz'],
                                    alpha_A0=pparams['alpha_A0'],
                                    alpha_alpham=pparams['alpha_alpham'],
                                    alpha_alphaz=pparams['alpha_alphaz'],
                                    beta_A0=pparams['beta_A0'],
                                    beta_alpham=pparams['beta_alpham'],
                                    beta_alphaz=pparams['beta_alphaz'])

        rgs = r200critz/2.
        cgs = rvirs/rgs
        ks,ukouts = generic_profile_fft(rhofunc,cgs,rgs[...,None],hcos.zs,hcos.ks,xmax,nxs)
        self.uk_profiles[name] = ukouts.copy()

In [ ]:
def rho_gas_generic_x(x,m200critz,z,omb,omm,rhocritz,
                    gamma=default_params['battaglia_gas_gamma'],
                    rho0_A0=battaglia_defaults[default_params['battaglia_gas_family']]['rho0_A0'],
                    rho0_alpham=battaglia_defaults[default_params['battaglia_gas_family']]['rho0_alpham'],
                    rho0_alphaz=battaglia_defaults[default_params['battaglia_gas_family']]['rho0_alphaz'],
                    alpha_A0=battaglia_defaults[default_params['battaglia_gas_family']]['alpha_A0'],
                    alpha_alpham=battaglia_defaults[default_params['battaglia_gas_family']]['alpha_alpham'],
                    alpha_alphaz=battaglia_defaults[default_params['battaglia_gas_family']]['alpha_alphaz'],
                    beta_A0=battaglia_defaults[default_params['battaglia_gas_family']]['beta_A0'],
                    beta_alpham=battaglia_defaults[default_params['battaglia_gas_family']]['beta_alpham'],
                    beta_alphaz=battaglia_defaults[default_params['battaglia_gas_family']]['beta_alphaz'],
):
    rho0 = battaglia_gas_fit(m200critz,z,rho0_A0,rho0_alpham,rho0_alphaz)
    alpha = battaglia_gas_fit(m200critz,z,alpha_A0,alpha_alpham,alpha_alphaz)
    beta = battaglia_gas_fit(m200critz,z,beta_A0,beta_alpham,beta_alphaz)
    # Note the sign difference in the second gamma. Battaglia 2016 had a typo here.
    return (omb/omm) * rhocritz * rho0 * (x**gamma) * (1.+x**alpha)**(-(beta+gamma)/alpha)

In [ ]:
def battaglia_gas_fit(m200critz,z,A0x,alphamx,alphazx):
    # Any factors of h in M?
    return A0x * (m200critz/1.e14)**alphamx * (1.+z)**alphazx